In [9]:
from typing_extensions import TypedDict

class State(TypedDict):
    message: str
    sentiment: str

In [10]:
def analyze_sentiment(state):
    print("---Analyzing Sentiment---")
    text = state["message"].lower()
    
    # Simple sentiment analysis
    if any(word in text for word in ["good", "great", "happy"]):
        return {"sentiment": "positive"}
    if any(word in text for word in ["bad", "sad", "terrible"]):
        return {"sentiment": "negative"}
    return {"sentiment": "neutral"}

def positive_response(state):
    print("---Positive Response---")
    return {"message": "Thank you for the positive feedback! 😊"}

def negative_response(state):
    print("---Negative Response---")
    return {"message": "We’re sorry to hear that. How can we improve? 😞"}

In [11]:
from typing import Literal

def sentiment_router(state) -> Literal["positive_response", "negative_response"]:
    if state["sentiment"] == "positive":
        return "positive_response"
    return "negative_response"

In [12]:
from langgraph.graph import StateGraph, START, END

# Initialize graph
builder = StateGraph(State)

# Add nodes
builder.add_node("analyze_sentiment", analyze_sentiment)
builder.add_node("positive_response", positive_response)
builder.add_node("negative_response", negative_response)

# Add edges
builder.add_edge(START, "analyze_sentiment")
builder.add_conditional_edges("analyze_sentiment", sentiment_router)
builder.add_edge("positive_response", END)
builder.add_edge("negative_response", END)

# Compile graph
graph = builder.compile()

In [13]:
# Invoke the graph with input
result = graph.invoke({"message": "I had a great experience!"})

print(result)

---Analyzing Sentiment---
---Positive Response---
{'message': 'Thank you for the positive feedback! 😊', 'sentiment': 'positive'}


In [ ]:
from typing_extensions import TypedDict, Annotated
from typing import Any
from langchain_core.messages import HumanMessage, AnyMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

import os
from dotenv import load_dotenv
from openai import AzureOpenAI


# Load environment variables from .env file
load_dotenv()

# Fetch API credentials from environment variables
api_key = os.getenv("AZURE_OPENAI_GPT_4O_API_KEY")
end_point = os.getenv("AZURE_OPENAI_GPT_4O_ENDPOINT")
api_version = os.getenv("AZURE_OPENAI_GPT_4O_API_VERSION")

# Initialize Azure OpenAI client
llm_client = AzureOpenAI(
    api_key=api_key,
    azure_endpoint=end_point,
    api_version=api_version,
)


# Define the state for LangGraph
class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


# Tool: Person Information
def get_person_details(person: str) -> str:
    """
    Retrieve the details of a person.
    """
    return f"{person} is working as a DevOps Engineer."


# Node function that decides between LLM and tool call
def azure_llm_node(state: MessagesState):
    user_message = state["messages"][-1]  # latest message

    # Special case: if user asks about a person → use the tool
    if "who is" in user_message.content.lower():
        person = user_message.content.split("Who is")[-1].strip().rstrip("?")
        reply = get_person_details(person)
        return {"messages": [HumanMessage(content=reply)]}

    # Otherwise → call GPT-4o
    response = llm_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an AI assistant."},
            {"role": "user", "content": user_message.content},
        ],
        max_tokens=256,
    )

    reply = response.choices[0].message.content
    return {"messages": [HumanMessage(content=reply)]}


# Build LangGraph
builder = StateGraph(MessagesState)
builder.add_node("azure_llm_node", azure_llm_node)

builder.add_edge(START, "azure_llm_node")
builder.add_edge("azure_llm_node", END)

graph = builder.compile()


# 🧪 Test Case 1: Person Information Tool
print("\n=== Test Case 1: Person Information Tool ===")
messages = graph.invoke({"messages": [HumanMessage(content="Who is Harsha?", name = "Harsha")]})
for msg in messages["messages"]:
    msg.pretty_print()

# 🧪 Test Case 2: Normal GPT-4o response
print("\n=== Test Case 2: Normal LLM ===")
messages = graph.invoke({"messages": [HumanMessage(content="Hello!") ]})
for msg in messages["messages"]:
    msg.pretty_print()



=== Test Case 1: Person Information Tool ===
================================ Human Message =================================
Name: Harsha

Who is Harsha?
================================ Human Message =================================

Harsha is working as a DevOps Engineer.

=== Test Case 2: Normal LLM ===
================================ Human Message =================================

Hello!
================================ Human Message =================================

Hello! 😊 How can I assist you today?


In [20]:
from typing_extensions import TypedDict, Annotated
from typing import Any
from langchain_core.messages import HumanMessage, AnyMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import json

# Load environment variables
load_dotenv()
api_key = os.getenv("AZURE_OPENAI_GPT_4O_API_KEY")
end_point = os.getenv("AZURE_OPENAI_GPT_4O_ENDPOINT")
api_version = os.getenv("AZURE_OPENAI_GPT_4O_API_VERSION")

llm_client = AzureOpenAI(
    api_key=api_key,
    azure_endpoint=end_point,
    api_version=api_version,
)

# --- Tool: Person Information ---
def get_person_details(person: str) -> str:
    return f"{person} is working as a DevOps Engineer."

# LangGraph state
class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# Node function with tool binding (function calling)
def azure_llm_node(state: MessagesState):
    user_message = state["messages"][-1]

    response = llm_client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": user_message.content}],
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "get_person_details",
                    "description": "Get information about a person",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "person": {"type": "string", "description": "Name of the person"},
                        },
                        "required": ["person"],
                    },
                },
            }
        ],
        tool_choice="auto",  # model decides
    )

    choice = response.choices[0]

    # Agar tool call mila
    if choice.finish_reason == "tool_calls":
        tool_call = choice.message.tool_calls[0]
        args = json.loads(tool_call.function.arguments)
        result = get_person_details(**args)
        return {"messages": [HumanMessage(content=result)]}

    # Normal LLM reply
    reply = choice.message.content
    return {"messages": [HumanMessage(content=reply)]}


# Build graph
builder = StateGraph(MessagesState)
builder.add_node("azure_llm_node", azure_llm_node)
builder.add_edge(START, "azure_llm_node")
builder.add_edge("azure_llm_node", END)
graph = builder.compile()

# 🧪 Test Case 1: Tool calling
print("\n=== Test Case 1: Person Information Tool ===")
messages = graph.invoke({"messages": [HumanMessage(content="Who is Harsha?")]})
for msg in messages["messages"]:
    msg.pretty_print()

# 🧪 Test Case 2: Normal conversation
print("\n=== Test Case 2: Normal LLM ===")
messages = graph.invoke({"messages": [HumanMessage(content="Hello, how are you?")]})
for msg in messages["messages"]:
    msg.pretty_print()



=== Test Case 1: Person Information Tool ===
================================ Human Message =================================

Who is Harsha?
================================ Human Message =================================

Harsha is working as a DevOps Engineer.

=== Test Case 2: Normal LLM ===
================================ Human Message =================================

Hello, how are you?
================================ Human Message =================================

Hello! I'm here and ready to help. How can I assist you today?
